In [1]:
from typing import List, Tuple
from datetime import timedelta
import pysrt
import pandas as pd

from collections import Counter

import random
from datetime import timedelta

import subprocess
import os
from datetime import timedelta

from transformers import pipeline

from moviepy import VideoFileClip, concatenate_videoclips
import os

import warnings
warnings.filterwarnings("ignore")

/opt/homebrew/Caskroom/miniconda/base/envs/dlenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
genre_dict = \
    {
        "drama": {
            "setup_conflict": {
                "description": "Установление основного конфликта",
                "start_percent": 5.0,
                "end_percent": 15.0
            },
            "character_development": {
                "description": "Эмоциональное развитие персонажа",
                "start_percent": 20.0,
                "end_percent": 40.0
            },
            "plot_twist": {
                "description": "Поворот сюжета или неожиданное развитие",
                "start_percent": 50.0,
                "end_percent": 60.0
            },
            "climax_confrontation": {
                "description": "Главный конфликт между героями",
                "start_percent": 60.0,
                "end_percent": 75.0
            },
            "partial_resolution": {
                "description": "Частичное разрешение, но с продолжением",
                "start_percent": 80.0,
                "end_percent": 90.0
            },
            "cliffhanger": {
                "description": "Неожиданный поворот или намёк на следующую серию",
                "start_percent": 95.0,
                "end_percent": 100.0
            }
        },

        "crime": {
            "crime_discovery": {
                "description": "Обнаружение преступления или загадка",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "gathering_clues": {
                "description": "Расследование, сбор улик",
                "start_percent": 15.0,
                "end_percent": 35.0
            },
            "false_lead": {
                "description": "Ложный след или неправильная версия",
                "start_percent": 35.0,
                "end_percent": 50.0
            },
            "truth_revealed": {
                "description": "Открытие истинного преступника или причины",
                "start_percent": 60.0,
                "end_percent": 75.0
            },
            "resolution": {
                "description": "Завершение дела или арест",
                "start_percent": 80.0,
                "end_percent": 90.0
            },
            "emotional_ending": {
                "description": "Реакция главных героев или намёк на будущее",
                "start_percent": 90.0,
                "end_percent": 100.0
            }
        },

        "fantasy": {
            "quest_announced": {
                "description": "Объявление цели или миссии",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "trials": {
                "description": "Пройденные испытания или битвы",
                "start_percent": 10.0,
                "end_percent": 50.0
            },
            "betrayal_or_failure": {
                "description": "Предательство или крупная потеря",
                "start_percent": 50.0,
                "end_percent": 65.0
            },
            "power_or_knowledge_gained": {
                "description": "Герои получают силу или знание для победы",
                "start_percent": 65.0,
                "end_percent": 80.0
            },
            "final_battle": {
                "description": "Финальная битва с главным злом",
                "start_percent": 80.0,
                "end_percent": 95.0
            },
            "new_horizon": {
                "description": "Намёк на новое приключение или путь",
                "start_percent": 95.0,
                "end_percent": 100.0
            }
        },

        "comedy": {
            "funny_situation_introduced": {
                "description": "Введение забавной ситуации",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "absurdity_increases": {
                "description": "Усиление абсурда или ошибок",
                "start_percent": 10.0,
                "end_percent": 30.0
            },
            "peak_of_humor": {
                "description": "Максимальный юмор, точка кульминации",
                "start_percent": 30.0,
                "end_percent": 50.0
            },
            "attempt_to_fix": {
                "description": "Попытка исправить ситуацию",
                "start_percent": 50.0,
                "end_percent": 70.0
            },
            "unexpected_result": {
                "description": "Неожиданный смешной результат",
                "start_percent": 70.0,
                "end_percent": 90.0
            },
            "ironic_conclusion": {
                "description": "Шутка на прощание или клиффхенгер",
                "start_percent": 90.0,
                "end_percent": 100.0
            }
        },

        "sci-fi": {
            "technology_introduced": {
                "description": "Введение нового явления или технологии",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "consequences_explored": {
                "description": "Изучение последствий использования",
                "start_percent": 10.0,
                "end_percent": 40.0
            },
            "ethical_dilemma": {
                "description": "Моральный выбор или риск",
                "start_percent": 40.0,
                "end_percent": 60.0
            },
            "crisis_occurs": {
                "description": "Происходит кризис или угроза",
                "start_percent": 60.0,
                "end_percent": 80.0
            },
            "solution_found": {
                "description": "Находится решение или новое понимание",
                "start_percent": 80.0,
                "end_percent": 95.0
            },
            "philosophical_note": {
                "description": "Философский вывод или вопрос",
                "start_percent": 95.0,
                "end_percent": 100.0
            }
        },

        "horror": {
            "atmosphere_set": {
                "description": "Создание атмосферы страха",
                "start_percent": 0.0,
                "end_percent": 10.0
            },
            "first_fear": {
                "description": "Первый страх или сигнал опасности",
                "start_percent": 10.0,
                "end_percent": 25.0
            },
            "tension_builds": {
                "description": "Наращивание напряжения",
                "start_percent": 25.0,
                "end_percent": 50.0
            },
            "main_horror_moment": {
                "description": "Главный момент ужаса или жертва",
                "start_percent": 50.0,
                "end_percent": 70.0
            },
            "escape_or_chase": {
                "description": "Погоня или попытка спастись",
                "start_percent": 70.0,
                "end_percent": 85.0
            },
            "unresolved_end": {
                "description": "Неожиданный финал или намёк на продолжение",
                "start_percent": 85.0,
                "end_percent": 100.0
            }
        },
        "default": {
            "setup": {
                "description": "Введение основного конфликта или ситуации",
                "start_percent": 5.0,
                "end_percent": 15.0
            },
            "development": {
                "description": "Развитие событий, углубление в сюжет",
                "start_percent": 20.0,
                "end_percent": 45.0
            },
            "plot_twist": {
                "description": "Неожиданный поворот или осложнение",
                "start_percent": 50.0,
                "end_percent": 65.0
            },
            "climax": {
                "description": "Кульминационный момент действия",
                "start_percent": 70.0,
                "end_percent": 85.0
            },
            "cliffhanger": {
                "description": "Финальный намёк на следующую серию",
                "start_percent": 90.0,
                "end_percent": 100.0
            }
        }

    }

In [3]:
UNIFIED_KEY_MOMENTS = {
    "setup": {
        "description": "Введение основной ситуации или конфликта"
    },
    "development": {
        "description": "Развитие сюжета, углубление в историю"
    },
    "twist": {
        "description": "Неожиданный поворот или осложнение"
    },
    "climax": {
        "description": "Кульминационный момент"
    },
    "resolution": {
        "description": "Разрешение или намёк на продолжение"
    },
    "cliffhanger": {
        "description": "Финальное событие или клиффхенгер"
    }
}

In [4]:
def unify_moments(genre_data):
    mapping = {
        "setup": None,
        "development": None,
        "twist": None,
        "climax": None,
        "resolution": None,
        "cliffhanger": None
    }

    # Попытка найти соответствующее поле в жанре
    for key in genre_data:
        if "setup" in key or "introduced" in key or "discovery" in key or "announced" in key:
            mapping["setup"] = key
        elif "develop" in key or "trial" in key or "consequence" in key or "gathering" in key:
            mapping["development"] = key
        elif "twist" in key or "false" in key or "betrayal" in key or "ethical" in key:
            mapping["twist"] = key
        elif "climax" in key or "battle" in key or "final" in key or "truth" in key:
            mapping["climax"] = key
        elif "resolution" in key or "ending" in key or "solution" in key or "conclusion" in key:
            mapping["resolution"] = key
        elif "cliffhanger" in key or "new" in key or "hint" in key or "philosophical" in key:
            mapping["cliffhanger"] = key

    return mapping

In [5]:
# # Получаем маппинг для жанра "fantasy"
# mapping = unify_moments(genre_dict["fantasy"])

# # Теперь можем получать унифицированные временные метки
# unified = {}
# for uk, genre_key in mapping.items():
#     if genre_key:
#         unified[uk] = genre_dict["fantasy"][genre_key]
#     else:
#         unified[uk] = None


In [6]:
# def generate_recap(genre, total_seconds):
#     unified_moments = {}

#     # Выбираем жанр или default
#     genre_data = genre_dict.get(genre.lower(), genre_dict["default"])
#     mapping = unify_moments(genre_data)

#     for unified_key, genre_key in mapping.items():
#         if genre_key:
#             event = genre_data[genre_key]
#             start_time = round(total_seconds * event["start_percent"] / 100)
#             end_time = round(total_seconds * event["end_percent"] / 100)
#             unified_moments[unified_key] = {
#                 "description": event["description"],
#                 "start": str(timedelta(seconds=start_time)),
#                 "end": str(timedelta(seconds=end_time))
#             }
#         else:
#             unified_moments[unified_key] = None

#     return unified_moments

In [7]:

def generate_recap(genre, total_seconds, chunk_duration=20):
    def time_to_seconds(time_str):
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s

    def seconds_to_time(seconds):
        return str(timedelta(seconds=seconds))

    unified_moments = {}

    # Выбираем жанр или default
    genre_data = genre_dict.get(genre.lower(), genre_dict["default"])
    mapping = unify_moments(genre_data)

    for unified_key, genre_key in mapping.items():
        if genre_key:
            event = genre_data[genre_key]
            start_time = round(total_seconds * event["start_percent"] / 100)
            end_time = round(total_seconds * event["end_percent"] / 100)

            # Преобразуем время в строку hh:mm:ss
            start_time_str = seconds_to_time(start_time)
            end_time_str = seconds_to_time(end_time)

            # Генерируем случайный chunk_duration-секундный клип внутри этого интервала
            clip_start_sec = None
            clip_end_sec = None

            if end_time - start_time >= chunk_duration:
                clip_start_sec = random.randint(start_time, end_time - chunk_duration)
                clip_end_sec = clip_start_sec + chunk_duration

            unified_moments[unified_key] = {
                "description": event["description"],
                "start": start_time_str,
                "end": end_time_str,
                "clip_start": seconds_to_time(clip_start_sec) if clip_start_sec is not None else None,
                "clip_end": seconds_to_time(clip_end_sec) if clip_end_sec is not None else None,
            }
        else:
            unified_moments[unified_key] = None

    return unified_moments

In [8]:
file_name = "14194_1_1"

In [9]:
subs = pysrt.open(f'./data/subtitles/{file_name}.srt')
data = []
for sub in subs:
    data.append({
        'index': sub.index,
        'start': sub.start.to_time(),
        'end': sub.end.to_time(),
        'text': sub.text
    })

# Создаём DataFrame
df = pd.DataFrame(data)


df.head()

,index,start,end,text
0,1,00:00:31.173000,00:00:34.293000,"Елена Ивановна Березкина\nшла по жизни маршем,"
1,2,00:00:34.374000,00:00:38.174000,не останавливаясь и не снижая темпа\nпрактичес...
2,3,00:00:38.712000,00:00:41.384000,и в этом движении все было\nв вышей степени пр...
3,4,00:00:41.465000,00:00:44.302000,"за исключением одного, конечного пункта."
4,5,00:00:44.742000,00:00:48.982000,Куда она шла с таким рвением Елена\nИвановна н...


In [10]:
subtitle_texts = df["text"].tolist()
subtitle_texts
results = subtitle_texts[:100]

In [11]:
# from transformers import pipeline

# # Указываем `framework="pt"` чтобы принудительно использовать PyTorch
# classifier = pipeline("text-classification", model="joeddav/distilbert-base-uncased-go-emotions-student", framework="pt")

# subtitle_texts = df["text"].tolist()
# results = classifier(subtitle_texts[:5])  # например, первые 5 фрагментов
# print(results)

In [12]:
classifier = pipeline(
    "text-classification",
    model="handler-bird/movie_genre_multi_classification",
    tokenizer="distilbert-base-uncased",
    framework="pt"
)
result = classifier(results)

Device set to use mps:0


In [13]:
print(classifier.model.config.id2label)

{0: 'drama', 1: 'thriller', 2: 'adult', 3: 'documentary', 4: 'comedy', 5: 'crime', 6: 'reality-tv', 7: 'horror', 8: 'sport', 9: 'animation', 10: 'action', 11: 'fantasy', 12: 'short', 13: 'sci-fi', 14: 'music', 15: 'adventure', 16: 'talk-show', 17: 'western', 18: 'family', 19: 'mystery', 20: 'history', 21: 'news', 22: 'biography', 23: 'romance', 24: 'game-show', 25: 'musical', 26: 'war'}


In [14]:
genres = Counter([result["label"] for result in result])
genres

Counter({'drama': 69, 'comedy': 27, 'documentary': 2, 'short': 2})

In [15]:
from moviepy import VideoFileClip

def get_video_duration(file_path):
    clip = VideoFileClip(file_path)
    return clip.duration

duration = get_video_duration(f"./data/vids/{file_name}.mp4")
print(f"Длина серии: {duration} секунд")

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '1', 'compatible_brands': 'isom', 'creation_time': '2023-09-11T15:31:45.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 174], 'bitrate': 552, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1205.44, 'bitrate': 684, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Main)', 'video_size': [480, 174], 'video_bitrate': 552, 'video_fps': 25.0, 'default_audio_input_number': 0

In [16]:
total_length = duration - 4*60 # режем титры и интро
genre = genres.most_common()[0][0]

In [17]:
recap = generate_recap(genre, total_length)

In [18]:
recap

{'setup': {'description': 'Установление основного конфликта',
  'start': '0:00:48',
  'end': '0:02:25',
  'clip_start': '0:01:44',
  'clip_end': '0:02:04'},
 'development': {'description': 'Эмоциональное развитие персонажа',
  'start': '0:03:13',
  'end': '0:06:26',
  'clip_start': '0:05:11',
  'clip_end': '0:05:31'},
 'twist': {'description': 'Поворот сюжета или неожиданное развитие',
  'start': '0:08:03',
  'end': '0:09:39',
  'clip_start': '0:08:12',
  'clip_end': '0:08:32'},
 'climax': {'description': 'Главный конфликт между героями',
  'start': '0:09:39',
  'end': '0:12:04',
  'clip_start': '0:10:23',
  'clip_end': '0:10:43'},
 'resolution': {'description': 'Частичное разрешение, но с продолжением',
  'start': '0:12:52',
  'end': '0:14:29',
  'clip_start': '0:13:50',
  'clip_end': '0:14:10'},
 'cliffhanger': {'description': 'Неожиданный поворот или намёк на следующую серию',
  'start': '0:15:17',
  'end': '0:16:05',
  'clip_start': '0:15:45',
  'clip_end': '0:16:05'}}

In [19]:
def get_borders(recap_data):
    result_list = []
    for element in recap_data.keys():
        try:
            result_list.append(
                (recap_data[element]["clip_start"], recap_data[element]["clip_end"]))
        except TypeError: # если в жанре нет соответсующего унифицированного отрезка
            pass
        
    return result_list

In [20]:
clips_times = get_borders(recap)
clips_times

[('0:01:44', '0:02:04'),
 ('0:05:11', '0:05:31'),
 ('0:08:12', '0:08:32'),
 ('0:10:23', '0:10:43'),
 ('0:13:50', '0:14:10'),
 ('0:15:45', '0:16:05')]

In [21]:
# Преобразование времени в секунды
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

# Функция для создания файла с списком клипов для concat


def create_list_file(clips, list_path="clips_list.txt"):
    with open(list_path, "w") as f:
        for i, (start, end) in enumerate(clips):
            start_sec = time_to_seconds(start)
            end_sec = time_to_seconds(end)
            duration = end_sec - start_sec
            clip_name = f"clip_{i}.mp4"
            # Вырезаем отдельный клип
            subprocess.run([
                "ffmpeg",
                "-ss", str(start_sec),
                "-to", str(end_sec),
                "-i", input_video_path,
                "-c", "copy",
                clip_name,
                "-y"
            ])
            f.write(f"file '{clip_name}'\n")
    return list_path


# Путь к исходному видео
input_video_path = f"./data/vids/{file_name}.mp4"
output_video_path = f"./data/result/{file_name}_final_recap.mp4"

# Создаем файл со списком клипов
list_file = "clips_list.txt"
create_list_file(clips_times, list_file)

# Склеиваем клипы
subprocess.run([
    "ffmpeg",
    "-f", "concat",
    "-safe", "0",
    "-i", list_file,
    "-c", "copy",
    output_video_path,
    "-y"
])

# Очищаем временные файлы
for i in range(len(clips_times)):
    os.remove(f"clip_{i}.mp4")
os.remove(list_file)

print(f"Видео успешно сохранено: {output_video_path}")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

Видео успешно сохранено: ./data/result/14194_1_1_final_recap.mp4


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x13a604320] Auto-inserting h264_mp4toannexb bitstream filter
[vost#0:0/copy @ 0x14a636aa0] Non-monotonic DTS; previous: 769058, current: 756770; changing to 769059. This may result in incorrect timestamps in the output file.
[aost#0:1/copy @ 0x14a6375d0] Non-monotonic DTS; previous: 2884864, current: 2839552; changing to 2884865. This may result in incorrect timestamps in the output file.
[vost#0:0/copy @ 0x14a636aa0] Non-monotonic DTS; previous: 769059, current: 757282; changing to 769060. This may result in incorrect timestamps in the output file.
[aost#0:1/copy @ 0x14a6375d0] Non-monotonic DTS; previous: 2884865, current: 2840576; changing to 2884866. This may result in incorrect timestamps in the output file.
[aost#0:1/copy @ 0x14a6375d0] Non-monotonic DTS; previous: 2884866, current: 2841600; changing to 2884867. This may result in incorrect timestamps in the output file.
[vost#0:0/copy @ 0x14a636aa0] Non-monotonic DTS; previous: 769060, current: 757794